In [2]:
% matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import glob

import datetime as dt

import gsw as sw
from mpl_toolkits.basemap import Basemap, cm, interp

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})



/Users/archie/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
def matlab2datetime(matlab_datenum):
    day = dt.datetime.fromordinal(int(matlab_datenum))
    dayfrac = dt.timedelta(days=matlab_datenum%1) - dt.timedelta(days = 366)
    return day + dayfrac

In [7]:
# import KE index data

kei = pd.read_csv('../data/length_delh_posi_RG_KEI_lines.asc', sep = '   ')


/Users/archie/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [16]:
kei = kei.dropna(axis=1, how='all')
kei.loc[(kei['Decimal Yr']>2010.999) & (kei['Decimal Yr']<2012.999)]

# find the day number for the cruises and get the position of the KE for each one

,Decimal Yr,Length,Strength,Position,Unnamed: 5,Unnamed: 6,KEI,Unnamed: 8,KEI(smoothed)
940,2011.016,1244.061,52.949,35.177,1.175,NaN,1.175,NaN,0.908
941,2011.036,1172.512,45.015,35.400,1.056,NaN,0.939,NaN,0.898
942,2011.055,1131.266,46.227,35.310,0.916,NaN,0.770,NaN,0.888
943,2011.074,1136.114,45.515,35.171,0.882,NaN,0.627,NaN,0.878
944,2011.093,1186.760,54.168,34.932,0.900,NaN,0.760,NaN,0.868
945,2011.112,1267.165,53.514,34.998,0.884,NaN,0.709,NaN,0.859
946,2011.132,1299.332,53.665,35.024,0.862,NaN,0.681,NaN,0.850
947,2011.151,1222.438,51.583,35.052,0.872,NaN,0.689,NaN,0.841
948,2011.170,1244.763,51.929,35.207,0.899,NaN,0.805,NaN,0.833
949,2011.189,1264.585,52.352,35.244,0.987,NaN,0.940,NaN,0.825


In [6]:
Tok1 = pd.read_csv(r'/Users/archie/Google Drive/o2ar_data/Tokyo1_vSept16.csv', sep = ',')
Tok1['density'] = sw.rho(Tok1['S'].values,Tok1['T'].values,0)-1000
#Tok1 = Tok1[(Tok1['Lon']>minlon) & (Tok1['Lon']<maxlon)]

Tok1['dlat'] = np.insert(np.diff(Tok1['Lat']), 0, 0)
Tok1 = Tok1[np.abs(Tok1['dlat'])<0.08]
# Tok1 = distance(Tok1).reset_index(drop = True)

matlab2datetime(Tok1['Timestamp'].values)

TypeError: only length-1 arrays can be converted to Python scalars